In [1]:
"""
Scraper Fundamentus — versão ajustada
- Rodar para lista de tickers (ex.: ["PETR4"])
- Retorna/mostra um DataFrame pandas (uma linha por ticker)
- Não grava em Google Sheets
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from IPython.display import display

HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# ---------------- utilitários ----------------

def parse_brazil_number(s):
    """Converte textos numéricos BR para float ou retorna string.
       Também lida com percentuais (retorna float representando o número, ex: '41,7%' -> 41.7)."""
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    st = str(s).strip()
    if st == "" or st == "-" or st.lower() in ("n/a","na"):
        return None
    st = st.replace("\xa0", " ").strip()
    # detectar percentual
    is_pct = False
    if "%" in st:
        is_pct = True
        st = st.replace("%", "")
    # limpar "R$" e espaços
    st = st.replace("R$", "").strip()
    # se contém letras longas (ex: "PN") -> não é número
    if re.search(r'[A-Za-z]{2,}', st) and not re.search(r'\d', st):
        return st
    # formatos:
    try:
        if "." in st and "," in st:
            num = float(st.replace(".", "").replace(",", "."))
        elif "," in st and "." not in st:
            num = float(st.replace(",", "."))
        elif "." in st and "," not in st:
            # likely thousand sep -> remove dots
            num = float(st.replace(".", ""))
        else:
            num = float(st)
        return float(num) if not is_pct else float(num)
    except:
        return st

def extract_label_value_pairs_from_tr(tr):
    """Emparelha células adjacentes num <tr> retornando lista de (label, value)."""
    cells = [c.get_text(" ", strip=True) for c in tr.find_all(['td','th'])]
    pairs = []
    i = 0
    n = len(cells)
    while i < n:
        label = cells[i].strip()
        if label == "":
            i += 1
            continue
        j = i + 1
        while j < n and cells[j].strip() == "":
            j += 1
        if j >= n:
            break
        value = cells[j].strip()
        label_norm = re.sub(r'[:\s]+$', '', label)
        pairs.append((label_norm, value))
        i = j + 1
    return pairs

# ---------------- mapping de labels ----------------
# ordem importante: padrões específicos (margem ebit) antes de padrões genéricos como 'ebit'
LABEL_MAP = [
    (r'^(papel|ticker)$', 'ticker'),
    (r'cotac', 'cotacao'),
    (r'data.*ult', 'data_ultima_cotacao'),
    (r'min 52', 'min_52_sem'),
    (r'max 52', 'max_52_sem'),
    (r'vol .*2m|vol .*med', 'vol_med_2m'),
    (r'^setor$', 'setor'),
    (r'^subsetor$', 'subsetor'),
    (r'valor de mercado|valor mercado', 'valor_mercado'),
    (r'valor da firma|valor firma', 'valor_firma'),
    (r'n(ro|º)|n(ro|º).*a[cç]oes|número.*a[cç]oes|nro.*a[cç]oes', 'nro_acoes'),
    (r'p\/l|p\.?\/l', 'pl'),
    (r'lpa', 'lpa'),
    (r'p\/vp|p\.?\/vp', 'p_vp'),
    (r'vpa', 'vpa'),
    (r'marg.*ebit', 'marg_ebit'),            # específico: Marg. EBIT (percentual)
    (r'p\/ebit|p\.?\/ebit', 'p_ebit'),
    (r'marg.*brut|margem.*bruta', 'marg_bruta'),
    (r'psr', 'psr'),
    (r'^ebit$', 'ebit'),                     # específico: EBIT valor monetário
    (r'p_?ativos|p\/ativos', 'p_ativos'),
    (r'marg.*liquida|margem.*liquida', 'marg_liquida'),
    (r'p[_\s]?cap[_\s]?giro|p cap giro', 'p_cap_giro'),
    (r'p[_\s]?ativ[_\s]?circ[_\s]?liq', 'p_ativ_circ_liq'),
    (r'roic', 'roic'),
    (r'div.*yield|dividend.*yield', 'dividend_yield'),
    (r'roe', 'roe'),
    (r'ev.*ebitda|ev / ebitda', 'ev_ebitda'),
    (r'liquidez corr|liquidez_corr|liquidez', 'liquidez_corr'),
    (r'ev.*ebit|ev / ebit', 'ev_ebit'),
    (r'div br.*patrim|div br patrim', 'div_br_patrim'),
    (r'cres.*rec|cres_rec_5a', 'cres_rec_5a'),
    (r'giro.*ativo|giro_ativos', 'giro_ativos'),
    (r'^ativo$', 'ativo'),
    (r'disponibilidades', 'disponibilidades'),
    (r'ativo circulante|ativo_circulante', 'ativo_circulante'),
    (r'div.*bruta', 'div_bruta'),
    (r'div.*l[ií]quida', 'div_liquida'),
    (r'patrim(o|ô)nio', 'patrimonio_liquido'),
    (r'receita liquida', 'receita_liquida_12m'),
    (r'lucro l[ií]quido', 'lucro_liquido_12m'),
    (r'oscila', 'oscilacoes'),
    (r'empresa', 'empresa'),
    (r'tipo', 'tipo'),
    (r'subsetor', 'subsetor'),
    (r'ultimos 12 meses|ultimos_12_meses', 'ultimos_12_meses'),
]

def normalize_label(label):
    lab = label.lower().strip()
    trans = str.maketrans("áàãâéêíóôõúüç","aaaaeeiooouuc")
    lab_no = lab.translate(trans)
    for pat, std in LABEL_MAP:
        if re.search(pat, lab_no):
            return std
    s = re.sub(r'[:\.\-\/\(\)]', ' ', lab_no)
    s = re.sub(r'[^0-9a-z\s]', '', s)
    s = re.sub(r'\s+', '_', s).strip('_')
    return s if s else label.lower()

# ---------------- parse da página ----------------

def parse_fundamentus_page(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    raw = {}
    # coletar pares originais
    for table in soup.find_all('table'):
        for tr in table.find_all('tr'):
            pairs = extract_label_value_pairs_from_tr(tr)
            for label, value in pairs:
                if not label:
                    continue
                # priorizar primeiro não vazio
                if label in raw:
                    if raw[label] in (None, "", "-") and value:
                        raw[label] = value
                else:
                    raw[label] = value

    # garantir captura explícita de 'Setor' se existir (usa label exato)
    # fazer busca em raw por chaves que são exatamente 'Setor' (caso-insens)
    sector_val = None
    for k in list(raw.keys()):
        if k.strip().lower() == 'setor':
            sector_val = raw[k]
            break
    # normalização
    normalized = {}
    for k, v in raw.items():
        std = normalize_label(k)
        parsed = parse_brazil_number(v) if std not in ('page_title','headline') else v
        normalized[std] = parsed

    # se 'setor' foi detectado explicitamente em raw, sobrescrever a normalização com seu valor
    if sector_val is not None:
        normalized['setor'] = parse_brazil_number(sector_val)

    # tratar Marg. EBIT e EBIT especificamente se apareceram como textos diferentes
    # procurar em raw label que contenha 'marg' e 'ebit' (caso exista)
    for k in raw.keys():
        kl = k.lower()
        if 'marg' in kl and 'ebit' in kl:
            normalized['marg_ebit'] = parse_brazil_number(raw[k])
        if kl.strip() == 'ebit' or re.fullmatch(r'ebit', kl, flags=re.I):
            normalized['ebit'] = parse_brazil_number(raw[k])

    return normalized

# ---------------- montar DataFrame e pós-processar ----------------

def get_many_tickers_fundamentus_df(tickers, pause=0.6):
    session = requests.Session()
    session.headers.update(HEADERS)
    rows = []
    cols_union = set()

    for tic in tickers:
        url = f"https://www.fundamentus.com.br/detalhes.php?papel={tic.upper()}"
        r = session.get(url, timeout=15)
        r.raise_for_status()
        info = parse_fundamentus_page(r.text)
        info['ticker'] = tic.upper()
        rows.append(info)
        cols_union.update(info.keys())
        sleep(pause)

    cols = ['ticker'] + sorted([c for c in cols_union if c != 'ticker'])
    df = pd.DataFrame(rows, columns=cols)

    # ----- excluir colunas solicitadas (inclui page_title) -----
    to_drop_tokens = {'empresa','oscilacoes','page_title','pagetitle','papel','dia','mes','30_dias','30dias','tipo','subsetor','ultimos_12_meses'}
    drop_cols = []
    for c in df.columns:
        cl = c.lower()
        for t in to_drop_tokens:
            if t in cl:
                drop_cols.append(c)
                break
    # remover colunas de anos >= 2020
    for year in range(2020, 2100):
        y = str(year)
        for c in df.columns:
            if y in c:
                drop_cols.append(c)
    # remover colunas relacionadas a "últimos 3 meses" se existirem (tokens comuns)
    for c in df.columns:
        if re.search(r'3\s*mes|ult.*3', c, flags=re.I):
            drop_cols.append(c)
    drop_cols = sorted(set(drop_cols))
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

        # ----- Conversão para milhões (mantendo decimais em indicadores) -----
    indicators = {
        'pl','p_vp','p_ebit','psr','dividend_yield','roe','roic','lpa','vpa',
        'p_ativos','p_cap_giro','p_ativ_circ_liq','ev_ebitda','ev_ebit',
        'liquidez_corr','div_br_patrim','cres_rec_5a','giro_ativos'
    }

    for col in df.columns:
        # tratar nro_acoes: inteiro completo
        if col == 'nro_acoes':
            df[col] = pd.to_numeric(df[col], errors='coerce').apply(lambda x: int(x) if pd.notna(x) else x)
            df[col] = df[col].astype(object)
            continue

        ser_num = pd.to_numeric(df[col], errors='coerce')
        if ser_num.notna().any():
            if col in indicators:
                # manter decimais; especial: p_ativ_circ_liq com 2 casas decimais
                if col == 'p_ativ_circ_liq':
                    df[col] = ser_num.apply(lambda x: round(float(x), 2) if pd.notna(x) else x)
                else:
                    df[col] = ser_num.apply(lambda x: float(x) if pd.notna(x) else x)
            elif col == 'ebit':
                # EBIT em milhões, inteiro, sem decimais e sem notação científica
                df[col] = ser_num.apply(
                    lambda x: int(round(x / 1_000_000)) if pd.notna(x) and abs(x) >= 1_000 else (
                        int(x) if pd.notna(x) else x
                    )
                )
                df[col] = df[col].astype(object)
            else:
                # converter grandes valores para milhões (arredondar)
                def conv(v):
                    if pd.isna(v):
                        return v
                    try:
                        if abs(v) >= 1_000_000:
                            return int(round(v / 1_000_000))  # convertido para milhões, sem decimais
                        if float(v).is_integer():
                            return int(v)
                        return float(v)
                    except:
                        return v
                df[col] = ser_num.apply(conv)
                df[col] = df[col].astype(object)
        else:
            df[col] = df[col].astype(object)

    # ----- Garantir setor correto: se 'setor' vazio e 'subsetor' existir, usar subsetor as fallback -----
    if 'setor' not in df.columns and 'subsetor' in df.columns:
        df['setor'] = df['subsetor']
    # prefer explicit 'setor' value if present; we already mapped explicit earlier

    # ----- Reordenar colunas (best-effort) -----
    desired_order = [
        "ticker","cotacao","data_ultima_cotacao","min_52_sem","max_52_sem","vol_med_2m",
        "setor","valor_mercado","valor_firma","nro_acoes","pl","lpa","p_vp","vpa","p_ebit",
        "marg_bruta","psr","marg_ebit","p_ativos","marg_liquida","p_cap_giro","p_ativ_circ_liq",
        "roic","dividend_yield","roe","ev_ebitda","liquidez_corr","ev_ebit","div_br_patrim",
        "cres_rec_5a","giro_ativos","ativo","disponibilidades","ativo_circulante","div_bruta",
        "div_liquida","patrimonio_liquido","receita_liquida_12m","ebit","lucro_liquido_12m"
    ]

    final_cols = []
    existing = list(df.columns)
    used = set()
    for want in desired_order:
        match = None
        if want in existing:
            match = want
        else:
            # substring match
            for c in existing:
                if c in used:
                    continue
                cl = c.lower()
                if want.replace('_',' ') in cl or all(tok in cl for tok in want.split('_') if tok):
                    match = c
                    break
        if match:
            final_cols.append(match)
            used.add(match)
    # append remaining columns
    for c in existing:
        if c not in used:
            final_cols.append(c)
    df = df.reindex(columns=final_cols)

    # ----- Exibição: formatar colunas de indicadores com decimais adequados -----
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 400)
    # formatting: keep floats showing meaningful decimals
    def fmt(x):
        if pd.isna(x):
            return ""
        if isinstance(x, int):
            return f"{x:,}"
        if isinstance(x, float):
            # itens que are indicators keep up to 2 decimals except when integer-like
            return f"{x:,.2f}".rstrip('0').rstrip('.') 
        return str(x)
    # We will not convert df values to strings globally; display will show numeric types.
    # But ensure p_ativ_circ_liq is float with 2 decimals (already rounded above)

    return df

# ---------------- Execução exemplo ----------------

if __name__ == "__main__":
    tickers = ["PETR4","VALE3"]
    df = get_many_tickers_fundamentus_df(tickers, pause=0.6)
    display(df)
    print("\nDataFrame (uma linha por ticker):")
    print(df.to_string(index=False))


,ticker,cotacao,data_ultima_cotacao,min_52_sem,max_52_sem,vol_med_2m,setor,valor_mercado,valor_firma,nro_acoes,pl,lpa,p_vp,vpa,p_ebit,marg_bruta,psr,marg_ebit,p_ativos,marg_liquida,p_cap_giro,p_ativ_circ_liq,roic,dividend_yield,roe,ev_ebitda,liquidez_corr,ev_ebit,div_br_patrim,cres_rec_5a,giro_ativos,ativo,disponibilidades,ativo_circulante,div_bruta,div_liquida,receita_liquida_12m,ebit,lucro_liquido_12m,ebit_ativo,patrim_liq,ult_balanco_processado
0,PETR4,29.84,24/10/2025,28.3,35.88,863,"Petróleo, Gás e Biocombustíveis",384600,704190,12888700000,4.97,6.00,0.96,30.97,1.87,49.1,0.78,41.7,0.33,15.8,-8.88,-0.60,18.9,17.4,19.4,2.52,0.76,3.43,0.93,6.2,0.42,1174890,51847,135859,371437,319590,493122,205467,77372,17.5,399222,30/06/2025
1,VALE3,61.73,24/10/2025,47.78,61.88,1035,Mineração,280193,346489,4539010000,9.68,6.38,1.31,47.19,4.11,34.2,1.34,32.5,0.57,13.3,16.48,-1.59,15.8,7.4,13.5,4.05,1.22,5.08,0.45,-8.1,0.42,493226,31087,95964,97383,66296,209597,68204,28954,13.8,214175,30/06/2025



DataFrame (uma linha por ticker):
ticker cotacao data_ultima_cotacao min_52_sem max_52_sem vol_med_2m                           setor valor_mercado valor_firma   nro_acoes   pl  lpa  p_vp   vpa  p_ebit marg_bruta  psr marg_ebit  p_ativos marg_liquida  p_cap_giro  p_ativ_circ_liq  roic  dividend_yield  roe  ev_ebitda  liquidez_corr  ev_ebit  div_br_patrim  cres_rec_5a  giro_ativos   ativo disponibilidades ativo_circulante div_bruta div_liquida receita_liquida_12m   ebit lucro_liquido_12m ebit_ativo patrim_liq ult_balanco_processado
 PETR4   29.84          24/10/2025       28.3      35.88        863 Petróleo, Gás e Biocombustíveis        384600      704190 12888700000 4.97 6.00  0.96 30.97    1.87       49.1 0.78      41.7      0.33         15.8       -8.88            -0.60  18.9            17.4 19.4       2.52           0.76     3.43           0.93          6.2         0.42 1174890            51847           135859    371437      319590              493122 205467             77372     